In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR

In [ ]:
file_path = r"datos_apartamentos_rent.xlsx"
df = pd.read_excel(file_path)
df = pd.DataFrame(df)

# Se crea una columna con el precio mensual
df["price_monthly"] = df.apply(lambda x: x["price"] * 4 if x["price_type"] == "Weekly" else x["price"], axis=1)

# Tarea 2

In [ ]:
# EXPLORACIÓN DE DATOS

# Extraer las comodidades de los inmueblde de la columna "amenities"
def get_unique_amenities(df, column="amenities"):
    unique_amenities = set()
    df[column].dropna().apply(lambda x: unique_amenities.update(x.split(",")))
    return list(unique_amenities)


In [ ]:
# Obtener lista sin duplicados
unique_amenities_list = get_unique_amenities(df)
print(unique_amenities_list)

In [ ]:
#Estadisticas Descriptivas

print(df[["price_monthly", "square_feet", "bathrooms", "bedrooms", "time"]].describe())
print(df[["state", "cityname", "category", "amenities"]].describe(include="object"))

In [ ]:
# Boxplot de Precio

plt.figure(figsize=(8, 6))


sns.boxplot(y=df["price_monthly"], palette="coolwarm")

plt.ylabel("Precio")
plt.title("Distribución del Precio de Alquiler")

plt.show()

In [ ]:
# Boxplot Square Feet vs. Categoría

df_apartment = df[df["category"] == "housing/rent/apartment"]
df_other = df[df["category"] != "housing/rent/apartment"]

# Categoría Apartment

plt.figure(figsize=(8, 6))


sns.boxplot(x=df_apartment["category"], y=df_apartment["square_feet"], palette="coolwarm")

plt.xlabel("Categoría del Anuncio")
plt.ylabel("Tamaño en Pies Cuadrados")
plt.title("Distribución del Tamaño para 'Apartment'")
plt.show()

# Demás categorias

plt.figure(figsize=(8, 6))


sns.boxplot(x=df_other["category"], y=df_other["square_feet"], palette="coolwarm")

plt.xlabel("Categoría del Anuncio")
plt.ylabel("Tamaño en Pies Cuadrados")
plt.title("Distribución del Tamaño para Otras Categorías")

plt.xticks(rotation=45)
plt.show()

In [ ]:
# Grafico Dispersión Precio vs. Square Feet

plt.figure(figsize=(10, 6))

#Se filtra el valor de 40000 pies cuadrados para mejor visualización
df_filtered = df[df["square_feet"] < 40000]

sns.scatterplot(x=df_filtered["square_feet"], y=df_filtered["price_monthly"], alpha=0.5)

plt.xlim(df_filtered["square_feet"].min(), df_filtered["square_feet"].max())
plt.ylim(df_filtered["price_monthly"].min(), df_filtered["price_monthly"].max())

plt.xlabel("Tamaño en Pies Cuadrados")
plt.ylabel("Precio")
plt.title("Relación entre Precio y Metraje")

plt.show()

In [ ]:
# Gráfico Ranking Estados

state_counts = df["state"].value_counts()

plt.figure(figsize=(12, 6))


sns.barplot(x=state_counts.index, y=state_counts.values, palette="viridis")

plt.xlabel("Estado")
plt.ylabel("Cantidad de Registros")
plt.title("Cantidad de Registros por Estado")
plt.xticks(rotation=90)  # Rotar etiquetas del eje para mejor visibilidad

plt.show()

In [ ]:
# Ranking Ciudades más costosas

#Se puede modificar el num de ciudades a mostrar en el apartado head
city_avg_price = df.groupby("cityname")["price_monthly"].mean().sort_values(ascending=False).head(30)

plt.figure(figsize=(12, 6))


sns.barplot(x=city_avg_price.values, y=city_avg_price.index, palette="coolwarm")

plt.xlabel("Precio Promedio ($)")
plt.ylabel("Ciudad")
plt.title("Top 10 Ciudades con el Precio Promedio más Alto")

plt.show()

In [ ]:
# Gráfico de Violin Square Feet por Estado

plt.figure(figsize=(12, 6))


sns.violinplot(x=df["state"], y=df["square_feet"], palette="muted")

plt.xlabel("Estado")
plt.ylabel("Tamaño en Pies Cuadrados")
plt.title("Distribución del Tamaño de Apartamentos por Estado")

plt.xticks(rotation=90) #Se rotan las etiquetas para mejor visualización
plt.show()

In [ ]:
#Se excluye el estado de NY para ver mejor la distribución de los demás estados
df_filtered_states = df[df["state"] != "NY"]

plt.figure(figsize=(12, 6))


sns.violinplot(x=df_filtered_states["state"], y=df_filtered_states["square_feet"], palette="muted")

plt.xlabel("Estado")
plt.ylabel("Tamaño en Pies Cuadrados")
plt.title("Distribución del Tamaño de Apartamentos por Estado (Sin NY)")

plt.xticks(rotation=90)
plt.show()

In [ ]:
# Boxplot Square Feet por Número de Habitaciones

plt.figure(figsize=(10, 6))


sns.boxplot(x=df["bedrooms"], y=df["square_feet"])

plt.xlabel("Número de Habitaciones")
plt.ylabel("Tamaño en Pies Cuadrados")
plt.title("Distribución del Tamaño por Número de Habitaciones")
plt.show()

In [ ]:
# Heatmap de Baños por número de Habitaciones
cross_tab = pd.crosstab(df["bathrooms"], df["bedrooms"])

plt.figure(figsize=(8, 6))


sns.heatmap(cross_tab, annot=True, fmt="d", cmap="coolwarm", linewidths=0.5)

plt.xlabel("Número de Habitaciones")
plt.ylabel("Número de Baños")
plt.title("Relación entre Número de Habitaciones y Baños")

plt.show()

In [ ]:
# Matriz de correlación (bathrooms, bedrooms, square_feet, price_monthly)

# Se seleccionan las variables para la matriz de correlación
numeric_cols = ['bathrooms', 'bedrooms', 'square_feet', 'price_monthly']
df_numeric = df[numeric_cols].dropna()  # Eliminar filas con valores nulos

# Calcular la matriz de correlación
correlation_matrix = df_numeric.corr()

# Visualizar la matriz de correlación
plt.figure(figsize=(8, 6))
sns.heatmap(correlation_matrix, annot=True, cmap="coolwarm", fmt=".2f", linewidths=0.5)
plt.title("Matriz de Correlación")
plt.show()